In [2]:
!pip install transformers
!pip install wget
!pip install GPUtil

     |████████████████████████████████| 481kB 2.7MB/s 
     |████████████████████████████████| 870kB 12.5MB/s 
     |████████████████████████████████| 3.1MB 20.0MB/s 
     |████████████████████████████████| 1.0MB 36.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=dbf673e4f16db4609c6fcd28f68200ded0318a5703e35bc1b529d7f532acb2b6
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=2cb27cdcb8d5de59356593b81b47598f1cd65b0cfa06bfd806a8f6845dbe54d7
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=6eb21f9caa09046a9e1d8ebc7b8d76c10d5e398f340d89d7ac3684efaf69a1f0
  Stored in directory: /root/.cache/pip/wheels/3d/77/0

In [3]:
import wget
import os
import json
import torch
import numpy as np
import pandas as pd
from time import time, sleep
from transformers import BertModel, BertTokenizer
from torch import nn
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import Dataset, DataLoader
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [0]:
#download the dataset
url = 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_distractor_v1.json'
if not os.path.exists('./hotpot_dev_distractor_v1.json'):
  wget.download(url, './hotpot_dev_distractor_v1.json')

In [0]:
#open the dataset
with open('hotpot_dev_distractor_v1.json', 'r', errors='ignore') as json_data:
  my_data = json.load(json_data, encoding='utf-8')

In [16]:
entry = my_data[0]
print(json.dumps(entry, indent=2))

{
  "_id": "5a8b57f25542995d1e6f1371",
  "answer": "yes",
  "question": "Were Scott Derrickson and Ed Wood of the same nationality?",
  "supporting_facts": [
    [
      "Scott Derrickson",
      0
    ],
    [
      "Ed Wood",
      0
    ]
  ],
  "context": [
    [
      "Ed Wood (film)",
      [
        "Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.",
        " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
        " Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast."
      ]
    ],
    [
      "Scott Derrickson",
      [
        "Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.",
        " He lives in Los Angeles, California.",
        " He is best known for direct